In [3]:
import pandas as pd
import numpy as np
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import torchtext

text_field = torchtext.data.Field(sequential=True,      # text sequence
                                  tokenize=lambda x: x, # because are building a character-RNN
                                  include_lengths=True, # to track the length of sequences, for batching
                                  batch_first=True,
                                  use_vocab=True)       # to turn each character into an integer index
i = 0
for line in open('sp3.txt'):
        #print(line)
        i = i+1
        if i>=10:
            break
fields = [('line', text_field)]
dataset = torchtext.data.TabularDataset("sp3.txt", # name of the file
                                        "csv",  # fields are separated by a tab
                                        fields)
split = [0.6, 0.2,0.2]
train, valid, test =  dataset.split(split, stratified=False, strata_field=None, random_state=None)
text_field.build_vocab(train)
# print(text_field.vocab.stoi)
# print(text_field.vocab.itos)
train_iter = torchtext.data.BucketIterator(train,
                                           batch_size=32,
                                           sort_key=lambda x: len(x.line), # to minimize padding
                                           sort_within_batch=True,        # sort within each batch
                                           repeat=False)                   # repeat the iterator for multiple epochs
for i, batch in enumerate(train_iter):
    if i >= 10:
        break
    lines = batch.line[0]
    #print(lines)

In [5]:
##a naive model following the lecture notes with 1 hidden layer - GRU 
class CartmanLSTM(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_classes):
        super(CartmanLSTM, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(vocab_size, hidden_size, batch_first = True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        
        #set intiial hidden state and cell state
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        c0 = torch.zeros(1, x.size(0), self.hidden_size)
        
        #forward propogate
        out, _ = self.rnn(x, (h0, c0))
        
        #passs output of last time step to classifier
        out = self.fc(out[:, -1, :])
        
        return out
        
    def init_hidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size)

In [6]:
def evaluate(model, prime_str='I', predict_len=300, temperature=0.8):
    
    hidden = model.init_hidden()
    prime_input = text_to_tensor(prime_str)
    predicted = prime_str
    
    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = int(torch.multinomial(output_dist, 1)[0])
        # Add predicted character to string and use as next input
        predicted_char = vocab_itos[top_i]
        predicted += predicted_char
        inp = text_to_tensor(predicted_char)

    return predicted

def train(model, num_iters=2000, lr=0.004):
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    for it in range(num_iters):
        # get training set
        inp, target = random_training_set()
        # cleanup
        optimizer.zero_grad()
        # forward pass
        hidden = model.init_hidden()
        output, _ = model(inp, hidden)
        loss = criterion(output, target)
        # backward pass
        loss.backward()
        optimizer.step()

        if it % 200 == 199:
            print("[Iter %d] Loss %f" % (it+1, float(loss)))
            print("    " + evaluate(model, ' ', 300))
